This tutorial demonstrates how to run an Ehrenfest dynamics simulation
for a simple polaritonic system.
- Author: Yuchen Wang

We will:
- Define model and dynamics parameters
- Initialize nuclear and electronic degrees of freedom
- Run the Ehrenfest dynamics in polaritonic basis
- Plot the resulting dynamics


In [ ]:
#Importing Required Libraries
import math
import numpy as np
import warnings
from functools import reduce

# Core simulation modules
from polariton import compute_model
from liblibra_core import *
import util.libutil as comn
from libra_py import units
from libra_py.models import Tully
import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot

# Specific dynamics recipes
from recipes import ehrenfest_adi_ld, ehrenfest_dia, ehrenfest_adi_nac, fssh

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')


In [ ]:
# Number of electronic states in the simulation
nstates = 4
# Initial electronic state index
n_init = 1

# Define polariton model parameters
model_params = {
    "model": 1,
    "model0": 1,
    "ndof": 1,          # Number of nuclear degrees of freedom
    "g_c": 0.005,       # Light-matter coupling
    "omega_c": 0.1,     # Cavity frequency (a.u.)
    "epsilon": 1.0,     # Dielectric constant
    "nstates": nstates
}


In [ ]:
# Main dynamics parameters dictionary
dyn_general = {
    "nsteps": 500,                   # Total time steps
    "ntraj": 2000,                   # Number of trajectories
    "nstates": nstates,              # Electronic states
    "dt": 0.1 * units.fs2au,         # Time step (in a.u.)
    "num_electronic_substeps": 1,    # Substeps per nuclear step
    "isNBRA": 0, "is_nbra": 0,       # Disable NBRA
    "progress_frequency": 0.1,       # Progress printing frequency
    "which_adi_states": range(nstates),
    "which_dia_states": range(nstates),
    "mem_output_level": 3,           # Memory output detail level
    "properties_to_save": [
        "timestep", "time", "q", "p", "f", "Cadi", "Cdia",
        "Epot_ave", "Ekin_ave", "Etot_ave",
        "se_pop_adi", "se_pop_dia",
        "sh_pop_adi", "sh_pop_dia",
        "mash_pop_adi", "mash_pop_dia"
    ],
    "prefix": "adiabatic_md",
    "prefix2": "adiabatic_md"
}

# Load the specific Ehrenfest recipe (adiabatic NACs)
# One can choose other receipes
# fssh.load(dyn_general)
#ehrenfest_adi_ld.load(dyn_general)
# ehrenfest_dia.load(dyn_general)
ehrenfest_adi_nac.load(dyn_general)

In [ ]:
# Initialization type for nuclear DOFs:
# 0: fixed coords and momenta
# 1: fixed coords, sampled momenta
# 2: sampled coords, fixed momenta
# 3: both coords and momenta sampled
# Here the paramters are set according to the J. Chem. Phys. 157, 104118 (2022) paper
icond_nucl = 3

nucl_params = {
    "ndof": 1,
    "q": [-4],                       # Initial position
    "p": [0.0],                      # Initial momentum
    "mass": [1836.0],                 # Proton mass in a.u.
    "force_constant": [0.000382**2 * 1836.0],
    "init_type": icond_nucl
}

In [ ]:
#Initial Conditions — Electronic DOFs
# Representation:
# 0: diabatic wavefunctions
# 1: adiabatic wavefunctions
rep = 0

# Initial population vector (all zeros except chosen initial state)
istates = [0.0] * nstates
istates[n_init] = 1.0

elec_params = {
    "verbosity": 2,
    "init_dm_type": 0,
    "ndia": nstates,
    "nadi": nstates,
    "rep": rep,
    "init_type": 0,
    "istates": istates,
    "istate": n_init
}


In [ ]:
# Running the Simulation
# Clone the general dynamics parameters for the specific run
dyn_params = dict(dyn_general)

# Create a random number generator for stochastic parts
rnd = Random()

# Run the Ehrenfest simulation
res = tsh_dynamics.generic_recipe(
    dyn_params,
    compute_model,
    model_params,
    elec_params,
    nucl_params,
    rnd
)


In [ ]:
#Plotting the Results
plot_params = {
    "prefix": "adiabatic_md",
    "filename": "mem_data.hdf",
    "output_level": 3,
    "which_trajectories": [0],
    "which_dofs": [0],
    "which_adi_states": list(range(nstates)),
    "which_dia_states": list(range(nstates)),
    "frameon": True,
    "linewidth": 3,
    "dpi": 300,
    "axes_label_fontsize": (8,8),
    "legend_fontsize": 8,
    "axes_fontsize": (8,8),
    "title_fontsize": 8,
    "what_to_plot": [
        "coordinates", "momenta",  "forces", "energies", "phase_space",
        "se_pop_adi", "se_pop_dia", "sh_pop_adi", "sh_pop_dia",
        "mash_pop_adi", "mash_pop_dia"
    ],
    "which_energies": ["potential", "kinetic", "total"],
    "save_figures": 1,
    "do_show": 1
}

tsh_dynamics_plot.plot_dynamics(plot_params)